In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import os
import pytz
from pytz import timezone
from scipy.stats import pearsonr

In [3]:
tz = timezone('US/Central') # This data uses GMT-6 rather than standard CST/CDT
def toUTC(d):
    return tz.normalize(tz.localize(d)).astimezone(pytz.utc)

In [27]:
def string2datetime(stringDTSeries):
    try:
        return [(dt.datetime.strptime(DT,'%Y/%m/%d %I:%M:%S %p')) for DT in stringDTSeries]
    except(ValueError):
        try:
            return [(dt.datetime.strptime(DT,'%Y/%m/%d %H:%M:%S')) for DT in stringDTSeries]
        except(ValueError):
            try:
                return [(dt.datetime.strptime(DT,'%m/%d/%Y %I:%M:%S %p')) for DT in stringDTSeries]
            except(ValueError):
                return [(dt.datetime.strptime(DT,'%m/%d/%Y %H:%M:%S')) for DT in stringDTSeries]

In [10]:
path = '/Volumes/RT&O/C Brace/GPS Analysis/'

In [12]:
os.listdir(path+'/UIC_GPSData')

['CBR08_Final_Summary_File.xlsx',
 'CBR11_Final_Summary_File.xls',
 'CBR15_Final_Summary_File.xlsx',
 'CBR16_Final_Summary_File.xlsx',
 'CBR19_Final_Summary_File.xlsx',
 'CBR21_Final_Summary_File.xlsx',
 'CBR22_Final_Summary_File.xlsx',
 'CBR24_Final_SummaryRLU.xlsx',
 'CBR25_Final_Summary_Rname.xlsx',
 'CBR26_Final_Summary.xlsx',
 'CBR27_Final_Summary_Rname.xlsx',
 'CBR28_Final_Summary_Rname.xlsx',
 'CBR30_Final_Summary_Rnames.xlsx',
 '~$CBR08_Final_Summary_File.xlsx']

## Add LandUse coordinates from GPS Data

In [15]:
Baseline = pd.read_excel(path+'/UIC_GPSData/CBR15_Final_Summary_File.xlsx')

In [16]:
Baseline.LandUse.unique()

array(['trip', 'COM_URBMIX', 'home', 'RES_SF_DETACHED', 'COM_CULT_ENT',
       'OS_REC', 'Non-Parcel Road ROW', 'TCU_PARKING', 'COM_REGIONAL',
       'COM_OFFICE', 'RES_MF', 'COM_BIGBOX', 'COM_MALL', 'COM_HOTEL',
       'INST_MEDICAL'], dtype=object)

In [17]:
Gpsdata = pd.read_csv(path+'GPSWT/CBR15_Baseline.csv')

In [28]:
Baseline.head(3)

,Date,Sequence,BeginTime,EndTime,Mode,LandUse,MeanSpeed,Numberpoints,TotalGap,Duration
0,2015-07-02,1,11:12:13,12:47:30,Car,trip,77.185558,518,9.116665,95.283333
1,2015-07-02,2,12:47:35,12:54:18,NaN,COM_URBMIX,2.076743,31,1.633333,6.716667
2,2015-07-02,3,12:54:23,13:00:53,Car,trip,33.312281,79,0.000000,6.500000


In [29]:
Gpsdata.head(3)

,Unnamed: 0,INDEX,TRACK ID,VALID,UTC DATE,UTC TIME,LOCAL DATE,LOCAL TIME,MS,LATITUDE,N/S,LONGITUDE,E/W,ALTITUDE,SPEED,HEADING,G-X,G-Y,G-Z,Wear/Non-Wear
0,0,1,1,FIXED,7/2/2015,16:12:13,7/2/2015,10:12:13,0,41.895511,N,87.618302,W,189.558014,0.0,0.0,0,0,0,Wear
1,1,2,1,FIXED,7/2/2015,16:12:14,7/2/2015,10:12:14,0,41.895511,N,87.618302,W,189.558014,0.0,0.0,0,0,0,Wear
2,2,3,1,FIXED,7/2/2015,16:12:15,7/2/2015,10:12:15,0,41.895511,N,87.618302,W,189.558014,0.0,0.0,0,0,0,Wear


In [30]:
x = Gpsdata['UTC DATE']+' '+Gpsdata['UTC TIME'] #full date/time
GpsDates = string2datetime(x) #convert to datetime object
GpsDates=pd.Series(GpsDates)

In [31]:
GpsDates[:10]

0   2015-07-02 16:12:13
1   2015-07-02 16:12:14
2   2015-07-02 16:12:15
3   2015-07-02 16:12:16
4   2015-07-02 16:12:17
5   2015-07-02 16:12:18
6   2015-07-02 16:12:21
7   2015-07-02 17:03:23
8   2015-07-02 17:03:28
9   2015-07-02 17:03:33
dtype: datetime64[ns]

In [35]:
loc_lat = []
loc_lon = []
wear_frac = []

for i in range(len(Baseline)):
    start = Baseline.BeginTime[i]
    end = Baseline.EndTime[i]
    d = Baseline.Date[i]
    d = d.strftime('%Y-%m-%d') 
    startdate = toUTC(dt.datetime.strptime(d+' '+start,'%Y-%m-%d %H:%M:%S'))
    enddate = toUTC(dt.datetime.strptime(d+' '+end,'%Y-%m-%d %H:%M:%S'))
       
    ind = (GpsDates > startdate) & (GpsDates < enddate)
    meanLat = np.mean(Gpsdata['LATITUDE'][ind])
    meanLon = np.mean(Gpsdata['LONGITUDE'][ind])
    loc_lat.append(meanLat)
    loc_lon.append(meanLon)
    
    wear_ind = (Gpsdata['Wear/Non-Wear']=='Wear') & ind
    wear_frac.append(sum(wear_ind)/sum(ind))
    
Baseline['Mean Lat'] = loc_lat
Baseline['Mean Lon'] = loc_lon
Baseline['Wear Frac'] = wear_frac


In [38]:
Baseline.head()

,Date,Sequence,BeginTime,EndTime,Mode,LandUse,MeanSpeed,Numberpoints,TotalGap,Duration,Mean Lat,Mean Lon,Wear Frac
0,2015-07-02,1,11:12:13,12:47:30,Car,trip,77.185558,518,9.116665,95.283333,41.716221,87.685847,1.000000
1,2015-07-02,2,12:47:35,12:54:18,NaN,COM_URBMIX,2.076743,31,1.633333,6.716667,41.528899,87.849158,1.000000
2,2015-07-02,3,12:54:23,13:00:53,Car,trip,33.312281,79,0.000000,6.500000,41.514940,87.850628,1.000000
3,2015-07-02,4,13:00:58,23:59:55,NaN,home,0.187761,7910,0.000000,658.950000,41.499339,87.857458,0.877213
4,2015-07-03,1,0:00:00,9:45:23,NaN,home,0.073735,7026,0.000000,585.383333,41.499315,87.857421,0.077449


In [39]:
Baseline.LandUse.unique()

array(['trip', 'COM_URBMIX', 'home', 'RES_SF_DETACHED', 'COM_CULT_ENT',
       'OS_REC', 'Non-Parcel Road ROW', 'TCU_PARKING', 'COM_REGIONAL',
       'COM_OFFICE', 'RES_MF', 'COM_BIGBOX', 'COM_MALL', 'COM_HOTEL',
       'INST_MEDICAL'], dtype=object)

# TODO:
* Loop through files and generate data for each brace (now sheets are renamed Baseline, SCO, CBR)